# LOAD TEXT

In [1]:
with open('text.txt', 'r', encoding='utf-8') as f:
	text = f.read().replace('\n', ' ')

In [2]:
len(text), len(text.split())

(37070, 6014)

# NLTK

In [3]:
from nltk import word_tokenize
tokenized_nltk = word_tokenize(text, language='russian')

# Stanza

In [4]:
import stanza
nlp = stanza.Pipeline(lang='ru', processors='tokenize', verbose=False)
tokenized_stanza = [tk.text for tk in nlp(text).iter_tokens()]

C:\Users\Андрей\AppData\Roaming\Python\Python310\site-packages\stanza\models\tokenization\trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.loa

# Razdel 

In [5]:
from razdel import tokenize
tokenized_razdel = [tk.text for tk in tokenize(text)]

# Segtok

In [6]:
from segtok.tokenizer import word_tokenizer
tokenized_segtok = word_tokenizer(text)

# Spacy

In [7]:
import spacy
nlp = spacy.load('ru_core_news_lg')
tokenized_spacy = [tk.text for tk in nlp(text) if tk.text != ' ']

# Pymorphy3

In [8]:
from pymorphy3.tokenizers import simple_word_tokenize
tokenized_pymorphy3 = simple_word_tokenize(text)

# Moses (not successful)

In [9]:
import mosestokenizer
with mosestokenizer.MosesTokenizer('ru') as tokenize:
    tokenized_moses = tokenize(text)

stdbuf was not found; communication with perl may hang due to stdio buffering.


# UDPipe

In [10]:
from ufal.udpipe import Model, Pipeline
model = Model.load('russian-ud-2.0-170801.udpipe')
pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, Pipeline.DEFAULT)
tokens = pipeline.process(text)
tokenized_ud = []
for i, sent in enumerate(tokens.split('\n\n')):
    for j, token in enumerate(sent.split('\n')):
        if (not token.startswith('#')) and token:
            tokenized_ud.append(token.split('\t')[1])

# Comparison

In [11]:
lens = [len(tokenized_nltk), len(tokenized_razdel), len(tokenized_segtok), len(tokenized_spacy), len(tokenized_pymorphy3), len(tokenized_moses), len(tokenized_ud)]
lens

[7288, 7287, 6737, 7354, 7307, 7348, 7132]

In [12]:
m = max(lens)
print(m)
print([m-l for l in lens])

7354
[66, 67, 617, 0, 47, 6, 222]


In [13]:
tokenized_nltk += ['' for i in range(m-len(tokenized_nltk))]
tokenized_razdel += [''  for i in range(m-len(tokenized_razdel))]
tokenized_segtok += [''  for i in range(m-len(tokenized_segtok))]
tokenized_spacy += [''  for i in range(m-len(tokenized_spacy))]
tokenized_pymorphy3 += [''  for i in range( m-len(tokenized_pymorphy3))]
tokenized_moses += [''  for i in range( m-len(tokenized_moses))]
tokenized_ud += [''  for i in range( m-len(tokenized_ud))]

In [14]:
print('{:20}\t{:20}\t{:20}\t{:20}\t{:20}\t{:20}\t{:20}'.format('nltk', 'razdel', 'segtok', 'spacy', 'pymorphy3', 'moses', 'ud'))
for tk in zip(tokenized_nltk, tokenized_razdel, tokenized_segtok, tokenized_spacy, tokenized_pymorphy3, tokenized_moses, tokenized_ud):
    print('{:20}\t{:20}\t{:20}\t{:20}\t{:20}\t{:20}\t{:20}'.format(*tk))

nltk                	razdel              	segtok              	spacy               	pymorphy3           	moses               	ud                  
Глава               	Глава               	Глава               	Глава               	Глава               	Глава               	Глава               
первая              	первая              	первая              	первая              	первая              	первая              	первая              
Через               	Через               	Через               	Через               	Через               	Через               	Через               
сто                 	сто                 	сто                 	сто                 	сто                 	сто                 	сто                 
лет                 	лет                 	лет                 	лет                 	лет                 	лет                 	лет                 
в                   	в                   	в                   	в                   	в                   	в            

In [15]:
import pandas as pd
df = pd.DataFrame(list(zip(tokenized_nltk, tokenized_razdel, tokenized_segtok, tokenized_spacy, tokenized_pymorphy3, tokenized_moses, tokenized_ud)), 
                  columns=['nltk', 'razdel', 'segtok', 'spacy', 'pymorphy3', 'moses', 'ud'])
df.to_excel('tokenizers_comparison.xlsx', index=False)